### Imports

In [31]:
import re
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import recomendaciones_escritoras.data.load as load

### Load data

In [32]:
escritoras_wiki = load.escritoras_wiki
data_processed = load.data_processed

### Read data

In [45]:
df = pd.read_csv(escritoras_wiki)
nombres = df['Nombre'].to_list()

### Scrape data

In [46]:
ids = []

for i in range(len(nombres)):

    params = {
    'utf8': '✓',
    'q': f'{nombres[i]}',
    'search_type': 'books',
    'search[field]': 'author'
    }

    response = requests.get('https://www.goodreads.com/search', params=params)

    # Return content of the response
    html = response.text

    # Parse html
    soup = bs(html, 'html.parser')
    
    try:

        # Look for <a> tags inside <div> tag
        results = [div.find('a', attrs={'class':'authorName'}) for div in soup.find_all('div', attrs={'class':'authorName__container'})]

        # Get first element from list of urls
        url_id = results[0].get('href')

        # Match author id from url
        author_id = re.findall(r'(\d+)', url_id)

        # Append value to list
        ids.append(author_id[0])
    
    except Exception:
        ids.append(np.nan)

### Process data

In [57]:
# Create dataframe from lists
df_ids = pd.DataFrame({'nombre': nombres, 'id': ids}, dtype=str)

df_ids

,nombre,id
0,Ophir Alviárez,24082699
1,Raiza Andrade,6102809
2,Laura Antillano,1482690
3,Ana Rosa Angarita Trujillo,24282274
4,Vivian Abenshushan,2347931
...,...,...
1111,Marta Susana Prieto,16379713
1112,Clementina Suárez,16965022
1113,Venus Ixchel Mejía,18869416
1114,Laura Acero,18937635


### Export data

In [56]:
# Sabe dataframe as csv
df_ids.to_csv(f'{data_processed}/goodreads_escritoras.csv', index=False)